Given some info about an a desired route, can we predict the airfare of the route?

In [ ]:
!wget https://raw.githubusercontent.com/Speeb04/SDSS-Datathon/refs/heads/main/Resources/Cases/Airline%20Tickets/final_output_data.csv

In [ ]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd             # For manipulating tabular data
import sklearn

Read in the CV and display the data

In [ ]:
data = pd.read_csv("final_output_data.csv.6")
data.describe()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import numpy as np

# Display data head and columns to confirm target variable and features
print("Data Head:")
display(data.head())
print("\nData Columns:")
print(data.columns)

From the column names and description, `TotalPerLFMkts_city1` is chosen as the target variable representing the fare price. The features are selected based on your description and available columns. Categorical features like `Year`, `quarter`, `citymarketid_1`, and `citymarketid_2` will be one-hot encoded.

In [ ]:
df = data.copy() # Use .copy() to avoid SettingWithCopyWarning
# Re-apply necessary initial cleaning steps, *before* one-hot encoding city1/city2
# 1. Drop columns with 100% missing values
missing_values_count = df.isnull().sum()
missing_values_percentage = (df.isnull().sum() / len(df)) * 100
missing_info = pd.DataFrame({
    'Missing Count': missing_values_count,
    'Missing Percentage': missing_values_percentage
})
columns_to_drop = missing_info[missing_info['Missing Percentage'] == 100].index.tolist()
df = df.drop(columns=columns_to_drop)


numerical_cols = [
    'nsmiles', 'passengers', 'large_ms', 'lf_ms',
    'fare', 'fare_lg', 'fare_low',
    'TotalFaredPax_city1', 'TotalPerLFMkts_city1', 'TotalPerPrem_city1',
    'TotalFaredPax_city2', 'TotalPerLFMkts_city2', 'TotalPerPrem_city2',
    'city1_gdp_2018', 'city2_gdp_2018', 'city1_pop_2024', 'city2_pop_2024', 'city1_gdppc', 'city2_gdppc'
]

# Standardize cleaning for all potential numerical columns
for col in numerical_cols:
    if col in df.columns:
        # Convert to string, replace problematic characters and '#DIV/0!', then convert to numeric
        df[col] = df[col].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
        df[col] = df[col].replace({'#DIV/0!': np.nan, 'nan': np.nan, 'None':np.nan})
        df[col] = pd.to_numeric(df[col], errors='coerce')


# Drop rows with NaN values in specified population columns
population_cols_to_check = ['city1_pop_2024', 'city2_pop_2024']
df.dropna(subset=[col for col in population_cols_to_check if col in df.columns], inplace=True)

# Impute any remaining NaNs after conversions for other numerical columns
for col in numerical_cols:
    if col not in population_cols_to_check and df[col].isnull().any():
        median_value = df[col].median()
        df[col] = df[col].fillna(median_value)

# Convert passengers to int after cleaning and imputation
df['passengers'] = df['passengers'].astype(int)

import seaborn as sns
print("Numerical Columns:", numerical_cols)
plt.figure(figsize=(12, 10))
sns.heatmap(df[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Matrix of Numerical Features')
plt.show()

# Now, create the 'route' column
df['route'] = df.apply(lambda row: '-'.join(sorted([row['city1'], row['city2']])), axis=1)

# Calculate the frequency of each route
route_counts = df['route'].value_counts()

In [ ]:
df['passengers']

In [ ]:
y = df['fare']
X = df.drop(columns=['fare', 'fare_lg', 'fare_low', 'citymarketid_1',
                     'citymarketid_2'])

X = X[['nsmiles', 'passengers', 'large_ms', 'lf_ms', 'Year', 'quarter','city1', 'city2', 'carrier_lg', 'carrier_low',
      'city1_gdp_2018', 'city2_gdp_2018', 'city1_pop_2024', 'city2_pop_2024', 'city1_gdppc', 'city2_gdppc',
      'route']]
X = pd.get_dummies(X, columns=['nsmiles', 'passengers', 'large_ms', 'lf_ms', 'Year', 'quarter','city1', 'city2', 'carrier_lg', 'carrier_low',
      'city1_gdp_2018', 'city2_gdp_2018', 'city1_pop_2024', 'city2_pop_2024', 'city1_gdppc', 'city2_gdppc',
      'route'], drop_first=True, dtype=int)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("First 5 rows of X:\n", X.head())
print("First 5 values of y:\n", y.head())

# # Convert categorical features to one-hot encoding
# X = pd.get_dummies(X, columns=['Year', 'quarter', 'citymarketid_1', 'citymarketid_2'], drop_first=True)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")




Now, let's train a Linear Regression model using the processed data.

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# Initialize and train the Linear Regression model
# model = DecisionTreeRegressor(criterion="squared_error", max_depth=60)
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
train_r2 = r2_score(y_train, model.predict(X_train))

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R2): {r2:.4f}")
print(f"Training R-squared: {train_r2:.4f}")
# plt.figure(figsize=(20, 10))
# plot_tree(model, feature_names=X.columns, filled=True, rounded=True, max_depth = 2)
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2) # Perfect prediction line
plt.xlabel('Actual Fare')
plt.ylabel('Predicted Fare')
plt.title('Actual vs. Predicted Fare Prices (Excluding fare_lg and fare_low)')
plt.grid(True)
plt.show()

We additionally want city attributes such as whether the city is a hub for full service airlines, the GDP of the city, population of the city, GDP p/c, etc.

For flights, we would also like to know which month of the year the flight occured.

Features:
Past fares for the route (continuous)

Quarter of the year (1-hot)

Distance (continuous)

Passenger count for the route (discrete)

Airline market share (continuous)

For each origin and destination:

City name (1-hot)

Is the city an air hub? (T/F)

GDP of the city (discrete)

Population (discrete)

GDP per capita (discrete)


In [ ]:
from sklearn.linear_model import LinearRegression
# Initialize the Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

print("Linear Regression model trained successfully.")


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Evaluation (after excluding fare_lg and fare_low):\n")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2) # Perfect prediction line
plt.xlabel('Actual Fare')
plt.ylabel('Predicted Fare')
plt.title('Actual vs. Predicted Fare Prices (Excluding fare_lg and fare_low)')
plt.grid(True)
plt.show()

In [ ]:
print(f"Intercept: {model.intercept_:.2f}")
print("Coefficients (first 10, for brevity):\n", model.coef_[:10])

# You can also get the coefficients mapped to their feature names
# for a more readable output. Note: X.columns contains all feature names.
coefficients_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print("\nAll Coefficients:\n")
display(coefficients_df.iloc[:20])

In [ ]:
display(coefficients_df.iloc[-20:])


Now, we get rid of the total covariates, and that is hindsight statistics.

In [ ]:
X = X.drop